In [37]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

# И класс терминального узла (листа)

class Leaf:
    counter = 0

    def __init__(self, data, labels):
        type(self).counter += 1
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        prediction = max(classes, key=classes.get)
        return prediction        



# Расчет критерия Джини

def gini(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity



def gain(left_labels, right_labels, root_gini):

    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)


def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

import time
# Построение дерева с помощью рекурсивной функции
def build_tree(data, labels, max_leafs, level=0):
    print(level)
    gain, t, index = find_best_split(data, labels)

    if gain == 0:
        
        return Leaf(data, labels)
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    #ИЗМЕНЕНИЯ также добавил переменных для ограничения по количеству листов, и измерения рекурсии
    if max_leafs != level:
        true_branch = build_tree(true_data, true_labels, max_leafs, level=level+1)

        false_branch = build_tree(false_data, false_labels, max_leafs, level=level+1)

    else :
        true_branch = Leaf(true_data, true_labels)
        false_branch = Leaf(false_data, false_labels)
    #ИЗМЕНЕНИЯ
    return Node(index, t, true_branch, false_branch)

def classify_object(obj, node):

    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [38]:
def print_tree(node, spacing=""):

    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction, '/n', node.counter)
        return

    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df_full = pd.read_csv('C:/Users/garic/Desktop/algos/lesson_4/data/cardio.csv', sep=';')

features = ['age', 'height']
target = ['cardio']

df = df_full[features + target]
print(df.shape)

train_data, test_data, train_labels, test_labels = train_test_split(df[features].values, 
                                                                    np.squeeze(df[target].values),
                                                                    test_size=0.3,
                                                                    random_state=1)


my_tree = build_tree(train_data, train_labels, 10)

print_tree(my_tree)

(70000, 3)
0
1
2
3
4
5
6
7
8
9
10
10
9
10
10
8
9
9
10
10
7
6
7
7
8
8
9
9
5
6
7
8
9
9
10
10
8
9
9
7
8
8
6
7
7
8
9
10
10
9
8
4
5
6
7
8
9
9
10
10
8
9
9
7
8
9
9
8
9
10
10
9
10
10
6
7
8
9
9
8
9
9
7
8
9
9
8
9
9
5
6
7
8
9
10
10
9
10
10
8
9
10
10
9
10
10
7
8
9
10
10
9
10
10
8
9
10
10
9
10
10
6
7
8
8
7
3
4
5
6
6
7
8
9
10
10
9
10
10
8
9
10
10
9
10
10
7
8
9
10
10
9
10
10
8
9
10
10
9
10
10
5
6
7
8
9
9
8
9
9
7
8
9
10
10
9
10
10
8
9
10
10
9
10
10
6
7
8
9
9
8
7
4
5
6
7
8
9
9
10
10
8
9
9
10
10
7
8
8
6
7
8
9
10
10
9
10
10
8
9
10
10
9
10
10
7
8
9
9
8
9
9
5
6
7
8
9
10
10
9
10
10
8
9
9
7
8
9
9
10
10
8
6
2
3
4
5
6
7
8
9
9
10
10
8
9
9
10
10
7
8
9
10
10
9
8
9
10
10
9
6
7
8
9
10
10
9
10
10
8
9
9
10
10
7
8
9
10
10
9
10
10
8
9
9
5
6
7
8
9
10
10
9
10
10
8
7
8
9
10
10
9
10
10
8
9
10
10
9
6
7
8
9
9
8
9
9
7
8
8
4
5
6
7
8
9
10
10
9
8
9
10
10
9
7
8
9
9
8
9
10
10
9
10
10
6
7
8
9
10
10
9
10
10
8
9
10
10
9
7
8
9
9
10
10
8
9
9
5
6
7
8
9
9
8
9
10
10
9
10
10
7
8
9
10
10
9
8
9
10
10
9
10
10
6
7
8
9
10
10
9
8
9
10
10
9
7
3
4

Я предположил, что каждая ступень рекурсии возвращает LEAF и для того, чтобы ограничиться по количеству листов, нужно ограничить глубину рекурсии, можно было попробывать через системное ограничение глубины рекурсии и обработку исключений, но мне кажется было бы не так наглядно